### Update bib records via Alma API

update several bib records through the alma api, replace the url in field 856$u and create field 024 with ark id.

The script reads the input file named 'SoSaGraphics_ARK_MMSIDs.json' (generated by other script) and loops through the list of ID-URL pairs. It downloads the bib record in marcxml, finds the 856 field, updates the url and creates an identifier field (024) and uploads the new bib record.
This script assumes there is only one field 856 $$u.

To run this script, add the following:

Write your Alma API key in a file named config.py in the same folder as your script. The file should look like this:

(This file is under .gitignore, the API keys are not publicly shared)


In [102]:
# api_key for SLSP NZ Sandbox or production

api_key_nz = "NZ_API_KEY"

# api_key for RZS Bib Sandbox or production

api_key_iz = "IZ_API_KEY"

This script assumes that field 856 contains the old links to the Sosa graphics (edoc.zhbluzern.ch) and can be overwritten (which is true for these ca. 2300 titles, but may have to be checked for another purpose). 

In [103]:
import requests
import urllib.request
import json
import xml.etree.ElementTree as ET
import re
import config

# Set your Alma API key, base API endpoint, input file name
api_key = config.api_key_nz
base_url = 'https://api-eu.hosted.exlibrisgroup.com/almaws/v1'
#filename = 'SoSaGraphics_ARK_MMSIDs_20230310.json'
filename = 'sosatest01.json'
#filename = 'sosatest10.json'

# Opening JSON file and return JSON object as a dictionary
f = open(filename)
id_url_list = json.load(f)
# debug: print(id_url_list)

# Loop through the list and update each bib record
for bib in id_url_list:
    nzid = bib['networkID']
    #print(nzid)
    ark = "https://n2t.net/"+bib['ARK']
    # Construct the API URL to get the bib record in MARCXML format
    bib_url = f'{base_url}/bibs/{nzid}?view=full&expand=None&apikey={api_key}'

    # Send the request to get the bib record
    response = requests.get(bib_url)

    # Parse the MARCXML data from the response content
    marcxml = ET.fromstring(response.content)  

    # Find the 856 field you want to replace
    for field in marcxml.findall('.//datafield[@tag="856"]'):
        # Find the subfield with the URL
        url_subfield = field.find('.//subfield[@code="u"]')
        if url_subfield is not None:
            # Replace the URL with the new value
            url_subfield.text = ark
            # debug: print(url_subfield.text)

    
    # Remove all 024 matching Subfield a with the given ARK
    '''
    marc_024 = marcxml.findall(".//datafield[@tag='024']")
    print(bib['ARK'])
    for field in marc_024:
        subfield_a = field.find(".//subfield[@code='a']")
        subfield_2 = field.find(".//subfield[@code='2']")
        print(subfield_a.text)
        print(subfield_2.text)
        if subfield_a.text == bib["ARK"] and subfield_2.text == "ark":
            print("match ARK")
            field.remove(subfield_a)
            field.remove(subfield_2)
    '''

    # Create a new 024 field with subfields a and 2
    # First check if there is a Field 024 matching the given ARK to avoid duplicate entries
    marc_024 = marcxml.findall(".//datafield[@tag='024']/subfield[@code='a']")
    match_024 = False
    for field in marc_024:
        if field.text == bib["ARK"]:
            print("ark match")
            match_024 = True
    if match_024 == False:
        record = marcxml.find("record")
        new_024 = ET.Element("datafield", attrib={"ind1": "7", "ind2": " ", "tag": "024"})
        new_a = ET.Element("subfield", attrib={"code": "a"})
        new_a.text = bib['ARK']
        new_2 = ET.Element("subfield", attrib={"code": "2"})
        new_2.text = 'ark'
        new_024.append(new_a)
        new_024.append(new_2)
        record.append(new_024)
        #debug: print(ET.tostring(new_024))
       

    # Convert the updated MARCXML back to a string
    updated_marcxml = ET.tostring(marcxml, encoding='utf-8')
    #debug: print(updated_marcxml)

    # Construct the API URL to update the bib record
    update_url = f'{base_url}/bibs/{nzid}?apikey={api_key}'

    # Send the request to update the bib record with the updated MARCXML
    response = requests.put(update_url, data=updated_marcxml, headers={'Content-Type': 'application/xml'})

    # Print the response status code to verify the update was successful
    print(f'Bib record {nzid} updated with URL {ark} - Response code: {response.status_code}')
    #debug: break

991134908649705501
Bib record 991134908649705501 updated with URL https://n2t.net/ark:/63274/zhb1h01r - Response code: 200
